[Reference](https://medium.com/firebird-technologies/building-sql-trainer-ais-backend-a-full-walkthrough-cea7789bda9f)

In [2]:
!pip install dspy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.9 MB/s eta 0:00:00


In [4]:
import dspy
lm = dspy.LM("openai/gpt-4o-mini", api_key="YOUR_OPENAI_API_KEY")
dspy.configure(lm=lm)

In [5]:
import dspy
# DSPy Signature for the create schema LLM program
class create_schema(dspy.Signature):
    """
    You are a schema generation assistant. Given a natural language description of the data or entities
    the user wants to store, generate a SQL CREATE TABLE schema that defines appropriate tables, columns,
    and data types. Use sensible names, appropriate data types, and include primary keys. If multiple tables
    are needed, include foreign key relationships where applicable. Return only the SQL schema.

    Example:
    User Prompt: "I want to store information about books, authors, and publishers. Each book has a title,
    publication year, genre, and is written by one or more authors. Each author has a name and birth year.
    Each publisher has a name and address."

    Output: A valid SQL schema containing CREATE TABLE statements to represent this data model.

    Your are using duckDB SQL, which is based on SQLite
    - DO NOT TRY to add foreign_key etc relationships


    """
    user_prompt = dspy.InputField(desc="The prompt the user has given on what schema they want you to generate")
    schema_sql = dspy.OutputField(desc="The SCHEMA SQL for the requested prompt")

schema_gen = dspy.Predict(create_schema)
schema_prompt = "Generate a schema for an online pet store"

response = schema_gen(user_prompt = schema_prompt)

print(response.schema_sql)

In [6]:
# DSPy signature that generates executable code that when executed will add data
class populate_table(dspy.Signature):
    """
    You are provided with a DuckDB SQL table schema.

    Your task is to write complete Python code that:
    - Uses DuckDB in Python.
    - Generates 250 rows of realistic simulated data based on column types and names.
    - Uses libraries such as `faker`, `random`, or `numpy` for data generation.
    - Creates the table using the exact schema provided.
    - Inserts the generated rows using DuckDB SQL INSERT statements (no DataFrame insertion).
    - Uses parameterized queries to avoid SQL injection and ensure clean formatting.
    - No need to import duckdb or connect it is already connected as conn
    - Do not do conn = duckdb.connect(), it is already connected
    - Take care of the foreign key relations, ensuring you add in good sequence!

    Do not return anything except the Python code.

    One-shot Example:

    Input
    table_schema = '''
    CREATE TABLE users (
        user_id INTEGER,
        full_name VARCHAR,
        email VARCHAR,
        age INTEGER,
        join_date DATE,
        is_active BOOLEAN
    );
    '''

    Output
    python_code = '''
    from faker import Faker
    import random
    from datetime import datetime, timedelta

    # Initialize
    fake = Faker()


    # Insert 250 rows
    insert_query = "INSERT INTO users VALUES (?, ?, ?, ?, ?, ?)"
    for i in range(1, 251):
        full_name = fake.name()
        email = fake.email()
        age = random.randint(18, 70)
        join_date = fake.date_between(start_date='-3y', end_date='today').isoformat()
        is_active = random.choice([True, False])
        conn.execute(insert_query, (i, full_name, email, age, join_date, is_active))
    '''
    """
    table_schema = dspy.InputField(desc="The DuckDB SQL schema for the table")
    python_code = dspy.OutputField(desc="Python code that generates simulated data & adds it via DuckDB SQL")

pop_table_gen = dspy.Predict(populate_table)

# Get the DDL for all tables in the database and print them
tables = [row[0] for row in conn.execute("SHOW TABLES").fetchall()]
schema_result = []
for table in tables:
    ddl = conn.execute(f"DESCRIBE {table}").fetchall()
    schema_result.append((table,ddl))
    print(f"Schema for table '{table}':\n{ddl}\n{'-'*40}")

response = populate_agent(table_schema = str(schema_result))

print(response.python_code)